In [1]:
import sys
from pathlib import Path

# Add parent directory to Python path
parent_dir = Path().resolve().parent
sys.path.insert(0, str(parent_dir))

In [2]:
%%capture
import logging
from io import BytesIO
from typing import Self, Optional
from datetime import datetime
import pyspark.pandas as pd
from src.spark_session import *

logging.getLogger("pyspark").setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)

In [3]:
spark = create_spark_session('neo')
spark.sparkContext.setLogLevel('ERROR')

25/06/08 17:29:15 WARN Utils: Your hostname, newlife resolves to a loopback address: 127.0.1.1; using 192.168.0.196 instead (on interface wlp0s20f3)
25/06/08 17:29:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/fastnnefarious/Projects/PycharmProjects/nasa_neo_pipeline/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fastnnefarious/.ivy2/cache
The jars for the packages stored in: /home/fastnnefarious/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-28b84b90-b75a-4562-8b8e-15fcec74cb88;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	

25/06/01 15:09:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/01 15:09:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Get Data

In [4]:
%%capture
df = spark.sql("SELECT * FROM iceberg_catalog.neo_db.gold_asteroids").toPandas()

# Configure Plots

In [5]:
import pandas as pd
import panel as pn
import hvplot.pandas


date_slider = pn.widgets.DateSlider(
    name='Select Date',
    start=df['observation_date'].min(),
    end=df['observation_date'].max(),
    value=df['observation_date'].min()
)


def miss_plot(selected_date):
    selected_data = df[df['observation_date'] == selected_date]
    print(selected_date)
    print(type(selected_date))
    return selected_data.hvplot.bar(
    x='name', y='miss_distance_ld',
    title='Miss Distance per Asteroid',
    color='orange', height=300, width=800
    )

def vel_hist(selected_date):
    selected_data = df[df['observation_date'] == selected_date]
    return selected_data.hvplot.hist(
        'velocity_kps', bins=10, title='Velocity Distribution', 
        height=300, width=800)
    

In [6]:
pn.Column(
    "# 🚀 Near Earth Objects Dashboard",
    date_slider,
    pn.bind(miss_plot,date_slider),
    pn.bind(vel_hist,date_slider)
)

2025-05-01
<class 'datetime.date'>


2025-05-17

Column
    [0] Markdown(str)
    [1] DateSlider(end=datetime.date(2025, 6, 1), name='Select Date', start=datetime.date(2025, 5, 1), value=datetime.date(2025, 5, 1))
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False)

# Dashboard

In [7]:
close_spark_session(spark)